In [1]:
import sys
import pandas as pd
import numpy as np

sys.path.append("../")
import biobss
import matplotlib.pyplot as plt
import neurokit2 as nk
from plotly_resampler import register_plotly_resampler

c:\Users\cagatay\miniconda3\envs\biobss_dev\lib\site-packages\plotly_resampler\aggregation\aggregators.py:24: UserWarning: Could not import lttbc; will use a (slower) python alternative.
  warnings.warn("Could not import lttbc; will use a (slower) python alternative.")


In [2]:
import importlib
import sys
import pandas as pd
import numpy as np

sys.path.append("../")
import biobss
import os
os.getcwd()
import neurokit2 as nk
sample_data = biobss.utils.load_sample_data("ECG")

In [3]:
sr=256

In [4]:
sample=np.array(sample_data[0]).flatten()

In [5]:
clean=biobss.pipeline.Bio_Process(
    process_method=biobss.ecgtools.filter_ecg,process_name="clean",argmap={"sampling_rate":"sampling_rate"},method="pantompkins")

find_peaks = biobss.pipeline.Bio_Process(process_method=biobss.ecgtools.ecg_peaks,
                                         process_name="rpeaks",returns_event=True,event_args={"event_name":"ECG_Peaks"})


pipeline = biobss.pipeline.Bio_Pipeline()

pipeline.process_queue.add_process(clean,input_signals="ECG_Raw",output_signals="ECG_Clean")
pipeline.process_queue.add_process(find_peaks,input_signals="ECG_Clean",output_signals="ECG_Peaks")

channel=biobss.pipeline.Bio_Channel(signal=sample,name="ECG_Raw",sampling_rate=sr)
pipeline.set_input(channel)


In [6]:
pipeline

Bio_Pipeline:
	Preprocessors: Process list:

	Processors: Process list:
	1: clean(ECG_Raw) -> ECG_Clean
	2: rpeaks(ECG_Clean) -> ECG_Peaks

	Postprocessors: Process list:

	Window Size(Seconds): Not Windowed
	Step Size: Not Windowed

In [7]:
# Create feature extraction steps

# Extract time domain features. input_signals dictionary contains feature prefixes as keys and input signals as values
# For example rms of EDA_Tonic is extracted as Tonic_rms in this case
Rpeak_features = biobss.pipeline.Feature(name="time_features", function=biobss.ecgtools.from_Rpeaks, input_signals=['ECG_Raw','Events_ECG_Raw'], sampling_rate=sr)


In [8]:
pipeline.run_pipeline()

In [9]:
pipeline.add_feature_step(Rpeak_features,input_signals=['ECG_Raw','Events_ECG_Clean'])

In [10]:
pipeline.extract_features()

In [11]:
pipeline.features

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
ecg_a_R,0.093000,0.372000,0.454000,0.125000,0.249000,0.134000,0.378000,0.389000,0.222000,0.175000,0.436000,0.343000,0.150000,0.080000,0.368000
ecg_RR0,0.648438,0.664062,-9.066406,0.648438,0.636719,0.640625,0.671875,0.656250,0.648438,0.636719,0.640625,0.644531,0.656250,0.632812,0.640625
ecg_RR1,0.664062,-9.066406,0.648438,0.636719,0.640625,0.671875,0.656250,0.648438,0.636719,0.640625,0.644531,0.656250,0.632812,0.640625,0.648438
ecg_RR2,-9.066406,0.648438,0.636719,0.640625,0.671875,0.656250,0.648438,0.636719,0.640625,0.644531,0.656250,0.632812,0.640625,0.648438,0.664062
ecg_RRm,-2.584635,-2.584635,-2.593750,0.641927,0.649740,0.656250,0.658854,0.647135,0.641927,0.640625,0.647135,0.644531,0.643229,0.640625,0.651042
ecg_RR_0_1,0.976471,-0.073244,-13.981928,1.018405,0.993902,0.953488,1.023810,1.012048,1.018405,0.993902,0.993939,0.982143,1.037037,0.987805,0.987952
ecg_RR_2_1,-13.652941,-0.071521,0.981928,1.006135,1.048780,0.976744,0.988095,0.981928,1.006135,1.006098,1.018182,0.964286,1.012346,1.012195,1.024096
ecg_RR_m_1,-3.892157,0.285078,-4.000000,1.008180,1.014228,0.976744,1.003968,0.997992,1.008180,1.000000,1.004040,0.982143,1.016461,1.000000,1.004016


In [16]:
ex=pipeline.data['ECG_Clean'].channel

In [30]:
pipeline

Bio_Pipeline:
	Preprocessors: Process list:

	Processors: Process list:
	1: clean(ECG_Raw) -> ECG_Clean
	2: rpeaks(ECG_Clean) -> ECG_Peaks

	Postprocessors: Process list:

	Window Size(Seconds): Not Windowed
	Step Size: Not Windowed

In [14]:
pk = pipeline.data['Events_ECG_Clean'].channel

In [25]:
tst=biobss.edatools.eda_statistical.get_stat_features(ex,prefix='test')

In [26]:
tst['test_mean']

0.00010641522711905394

In [28]:
f=lambda x: np.mean(x)

In [83]:
def foo(*args,**kwargs):
    print(args)
    print(kwargs)

In [84]:
foo()

()
{}


In [20]:
res=biobss.ecgtools.from_Rpeaks(ex,pk,256)

In [28]:
res

{0: {'ecg_a_R': 0.14941537935018082,
  'ecg_RR0': 0.6484375,
  'ecg_RR1': 0.6640625,
  'ecg_RR2': -9.06640625,
  'ecg_RRm': -2.5846354166666665,
  'ecg_RR_0_1': 0.9764705882352941,
  'ecg_RR_2_1': -13.652941176470588,
  'ecg_RR_m_1': -3.892156862745098},
 1: {'ecg_a_R': 0.1990699821912972,
  'ecg_RR0': 0.6640625,
  'ecg_RR1': -9.06640625,
  'ecg_RR2': 0.6484375,
  'ecg_RRm': -2.5846354166666665,
  'ecg_RR_0_1': -0.07324429125376992,
  'ecg_RR_2_1': -0.07152089616544594,
  'ecg_RR_m_1': 0.2850782708602614},
 2: {'ecg_a_R': 0.20239536323008903,
  'ecg_RR0': -9.06640625,
  'ecg_RR1': 0.6484375,
  'ecg_RR2': 0.63671875,
  'ecg_RRm': -2.59375,
  'ecg_RR_0_1': -13.981927710843374,
  'ecg_RR_2_1': 0.9819277108433735,
  'ecg_RR_m_1': -4.0},
 3: {'ecg_a_R': 0.1546145352183906,
  'ecg_RR0': 0.6484375,
  'ecg_RR1': 0.63671875,
  'ecg_RR2': 0.640625,
  'ecg_RRm': 0.6419270833333334,
  'ecg_RR_0_1': 1.01840490797546,
  'ecg_RR_2_1': 1.0061349693251533,
  'ecg_RR_m_1': 1.0081799591002045},
 4: {'ecg

In [25]:
import re 

In [26]:
def mytype(v):
    s = str(v)
    regex = re.compile(r'(?P<list>\[[^]]+\])|(?P<float>\d*\.\d+)|(?P<int>\d+)|(?P<string>[a-zA-Z]+)')
    return  r"<type '%s'>" % regex.search(s).lastgroup

In [29]:
mytype(res)

"<type 'int'>"

In [ ]:
def mytype(data):
    retun 